<a href="https://colab.research.google.com/github/emrunali/Clinical_report_based_on_VCF_data_and_ACMG_guidelines/blob/main/ACMG_compliant_clinical_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/WGLab/InterVar.git

fatal: destination path 'InterVar' already exists and is not an empty directory.


In [2]:
%cd InterVar

/content/InterVar


In [3]:
import sys
sys.path.append('/content/InterVar')

In [ ]:
#from Intervar import InterVar #InterVar class not defined within Intervar module

In [4]:
!git clone https://github.com/WGLab/doc-ANNOVAR.git

fatal: destination path 'doc-ANNOVAR' already exists and is not an empty directory.


In [5]:
!apt-get install python3-vcf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-vcf is already the newest version (0.6.8+git20170215.476169c-8build1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [6]:
import vcf
import gzip

In [7]:
vcf_file = "/content/normal_sample.deepvariant.vcf.gz"
with gzip.open(vcf_file, 'rt') as f: #not really required
  print(f.read(100)) #read first 1200 characters in the file, just verifying

##fileformat=VCFv4.2
##DeepVariant_version=1.5.0
##FILTER=<ID=LowQual,Description="Confidence in thi


In [8]:
vcf_reader = vcf.Reader(filename = vcf_file, compressed = True)
print(vcf_reader.metadata)

OrderedDict([('fileformat', 'VCFv4.2'), ('DeepVariant_version', ['1.5.0'])])


In [9]:
#would take longer as many variants
variant_data = [] #list of dictionaries, each dictionary has key:value pairs describing variant information from vcf file
for record in vcf_reader:
    variant = {
        'chr': record.CHROM[3:],
        'pos': record.POS,
        'ref': record.REF,
        'alt': str(record.ALT[0]),
    }
    variant_data.append(variant)

In [10]:
print(variant_data[:10])

[{'chr': 'r', 'pos': 12807, 'ref': 'C', 'alt': 'T'}, {'chr': 'r', 'pos': 13079, 'ref': 'C', 'alt': 'G'}, {'chr': 'r', 'pos': 13418, 'ref': 'G', 'alt': 'A'}, {'chr': 'r', 'pos': 13649, 'ref': 'G', 'alt': 'C'}, {'chr': 'r', 'pos': 13813, 'ref': 'T', 'alt': 'G'}, {'chr': 'r', 'pos': 13838, 'ref': 'C', 'alt': 'T'}, {'chr': 'r', 'pos': 14248, 'ref': 'T', 'alt': 'G'}, {'chr': 'r', 'pos': 14354, 'ref': 'C', 'alt': 'A'}, {'chr': 'r', 'pos': 14522, 'ref': 'G', 'alt': 'A'}, {'chr': 'r', 'pos': 14542, 'ref': 'A', 'alt': 'G'}]


In [12]:
intervar_instance = Intervar.InterVar()
classified_variants = intervar_instance.classify_variants(variant_data)

AttributeError: module 'Intervar' has no attribute 'InterVar'